In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
Combined_Data = pd.read_csv("Resources/Cleaned CSVs/Combined_Data.csv")

Combined_Data.drop('Tm', axis=1, inplace=True)

# Combined_Data.set_index('Tm', inplace=True)

In [3]:
corr_data = Combined_Data.corr()['playoffs']

low_corr_df = pd.DataFrame(corr_data.loc[(corr_data <= 0.2) & (corr_data >= -0.2)])
low_corr_df.rename(columns={'playoffs': 'correlation'}, inplace=True)

index_values = low_corr_df.index.tolist()

index_values

['#P',
 '3B',
 'BK',
 'BatAge',
 'CG_x',
 'CG_y',
 'CS',
 'Ch',
 'GDP',
 'GF',
 'HBP_x',
 'IBB_x',
 'IBB_y',
 'Rgood',
 'SB',
 'SH',
 'cSho']

In [4]:
columns_to_keep = ['#P',
 '3B',
 'BK',
 'BatAge',
 'CG_x',
 'CG_y',
 'CS',
 'Ch',
 'GDP',
 'GF',
 'HBP_x',
 'IBB_x',
 'IBB_y',
 'Rgood',
 'SB',
 'SH',
 'cSho','playoffs']

focused_df = Combined_Data[columns_to_keep]

focused_df

,#P,3B,BK,BatAge,CG_x,CG_y,CS,Ch,GDP,GF,HBP_x,IBB_x,IBB_y,Rgood,SB,SH,cSho,playoffs
0,30.0,50.0,5.0,29.2,1119.0,2.0,25.0,6150.0,110.0,160.0,52.0,36.0,43.0,14.0,79.0,38.0,1.0,0
1,35.0,29.0,8.0,27.3,1166.0,2.0,36.0,5889.0,99.0,160.0,66.0,53.0,43.0,-1.0,90.0,49.0,1.0,1
2,30.0,15.0,8.0,28.4,1112.0,2.0,22.0,5934.0,132.0,160.0,57.0,19.0,29.0,-6.0,81.0,13.0,0.0,0
3,23.0,31.0,3.0,27.7,1127.0,2.0,31.0,5819.0,130.0,160.0,55.0,38.0,8.0,-7.0,125.0,7.0,0.0,1
4,35.0,34.0,3.0,27.2,933.0,1.0,38.0,6208.0,107.0,162.0,78.0,67.0,33.0,7.0,66.0,40.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,28.0,12.0,7.0,27.5,1109.0,0.0,29.0,5944.0,122.0,162.0,70.0,17.0,6.0,3.0,101.0,13.0,0.0,0
146,40.0,24.0,1.0,26.8,1107.0,0.0,41.0,5802.0,109.0,162.0,95.0,7.0,10.0,-10.0,160.0,6.0,0.0,1
147,31.0,18.0,4.0,28.3,1109.0,3.0,19.0,5789.0,103.0,159.0,53.0,17.0,19.0,-1.0,79.0,10.0,1.0,1
148,27.0,19.0,12.0,28.8,1084.0,1.0,34.0,5733.0,129.0,161.0,69.0,14.0,29.0,3.0,99.0,4.0,1.0,1


 ## Separate the Features (X) from the Target (y)

In [5]:
y = focused_df["playoffs"]
X = focused_df.drop(columns="playoffs")

 ## Split our data into training and testing

In [6]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(112, 17)

 ## Create a Logistic Regression Model

In [7]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=1000,
                                random_state=1)
classifier

LogisticRegression(max_iter=1000, random_state=1)

 ## Fit (train) or model using the training data

In [8]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1)

 ## Score the model using the test data

In [9]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7410714285714286
Testing Data Score: 0.6052631578947368


 ## Make predictions

In [10]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,1,1
8,1,0
9,0,0


## Calculate the Accuracy Score

In [11]:
accuracy_score(y_test, predictions)

0.6052631578947368